In [138]:
# !pip install --upgrade --force-reinstall fastaiimport os
from pathlib import Path
import pandas as pd
from fastai.vision.all import *
from scipy.io import loadmat

In [139]:
# downloading images and delete false annotations file
path_data = untar_data(URLs.CARS, dest=Path.cwd())
path_annos = path_data/'cars_annos.mat'

# using try/except because of a bug in pathlib
try:
    path_annos.unlink(missing_ok=True)
except:
    pass

# downloading devkit containing true annotations
path_devkit = untar_data("https://ai.stanford.edu/~jkrause/cars/car_devkit.tgz", dest=path_data)

# defining train and test paths
path_train = path_data/'cars_train'
path_test = path_data/'cars_test'

In [140]:
# converting classes to brands only
classes = loadmat(path_devkit/'cars_meta.mat')
classes_brands = [str(x[0]).split(" ")[0] for x in classes['class_names'][0]]

# getting class labels for each file in train and saving
# filenames along with classes and labels into a dataframe
cars_train_annos = loadmat(path_devkit/'cars_train_annos.mat')
cars_train_files_labels = [(x[-1][0][:], x[-2][0][0])  for x in cars_train_annos['annotations'][0]]
train_labels_df = pd.DataFrame(data=cars_train_files_labels,
                               columns=['fname', 'cls'])
# adding brands as labels
train_labels_df['lbl'] = [classes_brands[i-1] for i in train_labels_df['cls'].values]

# defining a function which extracts class labels for each sample
def label_func(fname, labels_df=train_labels_df):
    return labels_df.loc[labels_df['fname']==fname.name, 'lbl'].values[0]

In [141]:
# Creating a DataBlock for the image data
cars = DataBlock(blocks = (ImageBlock, CategoryBlock),
                   get_items = get_image_files,
                   get_y = label_func,
                   splitter = RandomSplitter(),
                   item_tfms = Resize(224))

# Creating a DataLoaders object
dls = cars.dataloaders(path_train,
                       num_workers=0)

In [147]:
# Creating a Learner
learn = cnn_learner(dls, resnet34, metrics=error_rate)
learn.fine_tune(1)

RuntimeError: [enforce fail at ..\c10\core\CPUAllocator.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 2359296 bytes. Buy new RAM!